In [ ]:
#Shirley Cho
#CECS 456 01
#Machine Learning Final Project


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
os.environ['PYTHONHASHSEED']=str(123)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as python_random
from numpy.random import seed
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import sklearn
import numpy as np
np.random.seed(123) 
import matplotlib as mpl
import matplotlib.pyplot as plt

import cv2
from sklearn.model_selection import train_test_split

# loads ResNet50 trained weights from internet for transfer learning
from tensorflow.keras.applications.resnet50 import ResNet50


import pandas as pd 

# example of loading an image with the Keras API
from keras.preprocessing.image import load_img

#I used img_to_array to get my something x something x 3 for the pixels and features of the image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import matplotlib.image as mpimg

#for data agumentation


#I found random code to see if I could seed it to try to get same results each time
#But I think the gpu and other things in CNN make it so it will still be a bit random
seed(123)
np.random.seed(123) 
python_random.seed(123)
tf.random.set_seed(123)




from keras.preprocessing.image import ImageDataGenerator


#Code I found to try and make sure everything is seeded so that it's not very random each run? xD
def reset_seeds():
    np.random.seed(123) 
    python_random.seed(123)
    tf.random.set_seed(123)
    os.environ["PYTHONHASHSEED"] = str(123)
    

#File paths to extract the images - they get passed into getImageArrays(name) function as 'name' argument to return an array of 
#150x150x3
fileNameTrainN = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/NORMAL'
fileNameTrainP = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/PNEUMONIA'
fileNameValidN = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/NORMAL'
fileNameValidP = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA'
fileNameTestN ='/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/NORMAL'
fileNameTestP = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/PNEUMONIA'

#Function that takes the file path as an argument, finds all the files in that file path (which in this case are all images of chest xray)
#Then it will turn those images to arrays of their pixels using img_to_array() function.
#I also resize them to 150x150x3 because the RAM cannot handle how big these pictures are
#I think resizing may make us lose data a bit. I'm not sure how big an influence that is though
#Then I return an array of all the image files for this specific path
def getImageArrays(name):
    array = []
    for dirname, _, filenames in os.walk(name):
        for filename in filenames:
            if '.DS_Store' not in filename:
                temp = (img_to_array(load_img(os.path.join(dirname, filename)))/255.);
                resized = cv2.resize(temp, (150,150))
                array.append(resized)
            
    array = np.array(array)
    return array
 



#Assign the arrays of the image data
trainNormX1 = getImageArrays(fileNameTrainN)
trainPneX = getImageArrays(fileNameTrainP)
valNormX = getImageArrays(fileNameValidN)
valPneX = getImageArrays(fileNameValidP)
testNormX = getImageArrays(fileNameTestN)
testPneX = getImageArrays(fileNameTestP)

#y-label. 1 for pneumonia and 0 for normal lungs
pne = 1
norm = 0

#Make training normal lung data larger by duplicating it x3 here (I just concatenated same data over and over again)
trainNormX = np.concatenate((trainNormX1,trainNormX1),axis=0)
trainNormX = np.concatenate((trainNormX1,trainNormX),axis=0)

#y labels for each one (They are just an array of all 1's or 0's)
trainNormY = np.full(trainNormX.shape[0],norm)
trainPneY = np.full(trainPneX.shape[0],pne)
valNormY = np.full(valNormX.shape[0],norm)
valPneY = np.full(valPneX.shape[0],pne)
testNormY =  np.full(testNormX.shape[0],norm)
testPneY =  np.full(testPneX.shape[0],pne)

#Printed out to test
print("trainNormX")
print(trainNormX.shape)
print(trainNormX[0].shape) #150 x 150 x 3
plt.imshow(trainNormX[0])
plt.show()

#Image data agumentation modes
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

#Get the resnet50 model from online and get them already trained weights on imagenet. The input shape should be our image input's shape
res_model = tf.keras.applications.ResNet50(include_top = False,
                                    weights="imagenet",
                                    input_shape = (150,150,3))

#res_model.trainable = False

#Set most of the layers as false to freeze them. I only set the last 4 layers as trainable to finetune
for layer in res_model.layers[:171]:
    layer.trainable = False

#printed out to check if they were actually set to False
for i, layer in enumerate(res_model.layers):
    print(i,layer.name,"-",layer.trainable)
    
reset_seeds()    

#Building the model
cnn = tf.keras.models.Sequential() #initializing the CNN

#Added the resNet50 model that is frozen as part of our model
cnn.add(res_model)

#pooling later to decrease size
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Pooling 
cnn.add(tf.keras.layers.Flatten()) #Flaterning
cnn.add(tf.keras.layers.Dense(units=4096, activation="relu"))#Full Connection1
tf.keras.layers.Dropout(0.2)
cnn.add(tf.keras.layers.Dense(units=256, activation="relu"))#Full Connection2
tf.keras.layers.Dropout(0.2)
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))#Full Connection3
cnn.add(tf.keras.layers.Dense(units= 2, activation= "softmax"))#Output layer #I tried sigmoid with binary_crossentropy and it was much worse

#Use Adam with a learning rate of 10^-5 - lower learning rate to see if I could get better optimization ; more specific learning even if slower
opt = keras.optimizers.Adam(learning_rate=0.00001)
cnn.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

res_model.summary()
cnn.summary()

#Splitting the training and pneumonia for validation 
#(I split seperately to make sure that my validation had almost 50/50 of penumonia/normal instead of completely random)
X_trainN, X_valN, y_trainN, y_valN = train_test_split(trainNormX, trainNormY, test_size=0.1, random_state = 123) #Split train and val for normal lungs
X_trainP, X_valP, y_trainP, y_valP = train_test_split(trainPneX, trainPneY, test_size=0.1, random_state = 123) #Split train and val for penumonia lungs

#I'm adding normal and penumonia together for training
X = np.concatenate((X_trainN,X_trainP),axis=0)
y = np.concatenate((y_trainN,y_trainP),axis=0)

#I'm adding the extra 16 pneumonia/normal images from the given validation test set into training because why not
X = np.concatenate((X,valPneX),axis=0)
y = np.concatenate((y,valPneY),axis=0)

print(X.shape)
#print(X)
print("----------")
print(y.shape)
# print(y)

#I'm adding normal and pneumonia together for validation 
X_val = np.concatenate((X_valN,X_valP),axis=0)
y_val = np.concatenate((y_valN,y_valP),axis=0)

train_datagen.fit(X)

#it = train_datagen.flow(X, y,batch_size = 10)

# generate image of augmented image to see what it looks like
#batch = it.next()
for X_batch, y_batch in train_datagen.flow(X, y, batch_size=10):
    # Show 9 images
    for i in range(0, 10):
        plt.imshow(X_batch[i].reshape(150, 150, 3))
    plt.show()
    break
    
    # show the plot
# show the figure
    

#TRAINING MODEL on the split training set with validation set; shuffle set to True because I never shuffled any of my data
#I chose to train 5 epochs on the regular not data agumented data
cnn.fit(X, y, batch_size = 80, 
                            epochs=5, 
                            validation_data =(X_val,y_val),
                            shuffle = True
                           )
#Then I chose to train the model on 11 epochs of the agumented data to avoid over fitting 
#It should be agumented on the fly
#Validation set is the same and not agumented
history = cnn.fit(train_datagen.flow(X, y, batch_size = 80), steps_per_epoch=X.shape[0]//80,
                            epochs=20, 
                            validation_data =(X_val,y_val),
                            validation_steps = X_val.shape[0]//80,
                            shuffle = True)

#Creating the test set after training the model - Just used the given test set from the files
#Just adding the normal and pneumonia togerther though
X_test = np.concatenate((testNormX,testPneX),axis = 0)
y_test = np.concatenate((testNormY,testPneY),axis = 0)

print("Evaluation on test set")
score = cnn.evaluate(X_test, y_test)


X_new = testNormX[:10]
y_proba = cnn.predict(X_new)
print(y_proba)
print(y_proba.round(2))

y_pred = np.argmax(cnn.predict(X_new), axis=-1)
print(y_pred)

X_new_ = testPneX[:10]
y_proba_ = cnn.predict(X_new_)
print(y_proba_)
print(y_proba_.round(2))

y_pred_ = np.argmax(cnn.predict(X_new_), axis=-1)
print(y_pred_)


print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

print("--TEST:--")
#Test set confusion matrix and results
y_pred = np.argmax(cnn.predict(X_test), axis=-1)
print("-y_pred-")
print(y_pred)
confusionMatrix = confusion_matrix(y_test,y_pred)

print("Confusion matrix:")
print(confusionMatrix)
labels = ["Normal","Pneumonia"]
disp = ConfusionMatrixDisplay(confusion_matrix=confusionMatrix, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

acc = accuracy_score(y_test,y_pred)
print("Accuracy score:")
print(acc)

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# Print f1, precision, and recall scores
print("precision score: ")
print(precision_score(y_test, y_pred , average="macro"))
print("recall score: ")
print(recall_score(y_test, y_pred , average="macro"))
print("f1 score: ")
print(f1_score(y_test, y_pred , average="macro"))


print("--VAL:--")
#Validation Set information results and confusion matrix
y_pred = np.argmax(cnn.predict(X_val), axis=-1)
confusionMatrix = confusion_matrix(y_val,y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=confusionMatrix, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

acc = accuracy_score(y_val,y_pred)
print("Accuracy score:")
print(acc)


# Print f1, precision, and recall scores
print("precision score: ")
print(precision_score(y_val, y_pred , average="macro"))
print("recall score: ")
print(recall_score(y_val, y_pred , average="macro"))
print("f1 score: ")
print(f1_score(y_val, y_pred , average="macro"))

